In [ ]:
import pandas as pd
import numpy as np
import psycopg2

In [ ]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port=5433)
print("connection successful")
# cur = conn.cursor()
query="""SELECT * FROM eur_usd_final ORDER BY datetime DESC;"""
# cur.execute(query=query)
# data = cur.fetchall()
# cur.close()
df = pd.read_sql(query, conn)
conn.close()
print(df.head())

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=["id"])
df["tomorrow"] = df["close"].shift(1)
df = df.dropna()
df["target"] = (df["close"] < df["tomorrow"]).astype(int)
df = df.set_index("datetime")
df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=42)

test = df.iloc[:100]
train = df.iloc[100:]

predictors = ["close", "open", "high", "low"]
model.fit(train[predictors], train["target"])

In [ ]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)

In [ ]:
precision_score(test["target"], preds)

In [ ]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=42)

test_data = df.iloc[:500]
train_cv_data = df.iloc[500:]

predictors = df.columns.drop(["target", "tomorrow"])

In [ ]:
predictors

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import precision_score

tscv = TimeSeriesSplit(
    n_splits=5,
    test_size=400,
    gap=1
)

X = train_cv_data[predictors]
y = train_cv_data["target"]

precision_scores = []

for i, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(
        X=X_tr,
        y=y_tr
    )

    preds = model.predict(X_val)
    fold_score = precision_score(y_val, preds)
    precision_scores.append(fold_score)

    print(f"Fold {i+1} Precision Score: {fold_score:.4f}") 

print(f"\n✅ Avg CV Precision Score: {np.mean(precision_scores):.4f} ± {np.std(precision_scores):.4f}")


In [ ]:
test_predictions = model.predict(test_data[predictors])

test_score = precision_score(test_predictions, test_data["target"])

In [ ]:
test_score

In [ ]:
import joblib
joblib.dump(model, 'forex_model.pkl')

In [ ]:
import psycopg2
import pandas as pd 

conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port=5433)
print("connection successful")
query="""SELECT * FROM eur_usd_train ORDER BY datetime DESC;"""
df = pd.read_sql(query, conn)
conn.close()
print(df.head())

In [ ]:
df.head()

In [ ]:
df = df.set_index("datetime")
df 

In [ ]:
df = df.sort_index(ascending=True)
df

In [ ]:
df = df.drop(columns=["id"])
df

In [ ]:
df.to_csv("train_sample.csv")

# Parameter Tuning

In [1]:
import pandas as pd
df = pd.read_csv("train_sample.csv", index_col=["datetime"])
df.head()

,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
datetime,,,,,,,,,,,
2010-09-10,1.26899,1.27444,1.26470,1.26931,0.998470,0.993735,0.993757,0.933419,0.915486,1.27170,1
2010-09-13,1.27189,1.28879,1.27170,1.27170,1.000941,0.998375,0.995108,0.935701,0.917197,1.28720,1
2010-09-14,1.28717,1.30180,1.28329,1.28720,1.006057,1.010486,1.006453,0.947585,0.928355,1.29917,1
2010-09-15,1.29911,1.30359,1.29592,1.29917,1.004628,1.014885,1.014923,0.956866,0.936955,1.30151,1
2010-09-16,1.30130,1.31130,1.29789,1.30151,1.000900,1.012235,1.015854,0.959013,0.938611,1.30760,1


In [2]:
X = df.drop(columns=["tomorrow", "target"])
y = df["target"]

In [4]:
from src.model_factory import ModelFactory
classifier = ModelFactory.get_model('random_forest', 'random_forest')
param_space = classifier.param_space

In [8]:
def suggest_params(trial, param_space: dict):
    params = {}
    for name, spec in param_space.items():
        kind = spec[0]

        if kind == 'int':
            _, low, high = spec
            params[name] = trial.suggest_int(name, low, high)

        elif kind == "float":
            _, low, high = spec
            params[name] = trial.suggest_float(name, low, high)
        
        elif kind == "categorical":
            _, choices = spec
            params[name] = trial.suggest_categorical(name, choices)

        else:
            raise ValueError(f"Unknown param type: {kind}")
    
    return params

In [9]:
params = suggest_params(trial, param_space)
params

NameError: name 'trial' is not defined

In [ ]:
import optuna 
from src.model_factory import ModelFactory
from src.utility import cross_validate_model

def objective_function(trial):
    classifier = ModelFactory.get_model("random forest", "random_forest")
    params = suggest_params(trial, classifier.param_space)
    classifier.build_model(params)

    mean_score, std_score = cross_validate_model(classifier, X, y)

    return mean_score


In [11]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective_function, n_trials=50)

[I 2026-01-15 13:00:35,725] A new study created in memory with name: no-name-1036b293-6211-4749-a8c2-bdcacd545382
[I 2026-01-15 13:00:36,837] Trial 0 finished with value: 0.5495165148538648 and parameters: {'n_estimators': 56, 'max_depth': 11, 'min_samples_split': 2}. Best is trial 0 with value: 0.5495165148538648.
[I 2026-01-15 13:00:38,658] Trial 1 finished with value: 0.562620075887958 and parameters: {'n_estimators': 88, 'max_depth': 13, 'min_samples_split': 5}. Best is trial 1 with value: 0.562620075887958.
[I 2026-01-15 13:00:42,736] Trial 2 finished with value: 0.5725063640606163 and parameters: {'n_estimators': 195, 'max_depth': 13, 'min_samples_split': 2}. Best is trial 2 with value: 0.5725063640606163.
[I 2026-01-15 13:00:44,957] Trial 3 finished with value: 0.5697244497244498 and parameters: {'n_estimators': 111, 'max_depth': 12, 'min_samples_split': 5}. Best is trial 2 with value: 0.5725063640606163.
[I 2026-01-15 13:00:47,198] Trial 4 finished with value: 0.570207962363708

In [12]:
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.5901090756939814
Best hyperparameters: {'n_estimators': 165, 'max_depth': 14, 'min_samples_split': 4}
